In [41]:
import sys
import eastern
import pytz
import numpy as np
import pandas as pd
import shinybroker as sb
import statsmodels.api as sm
from datetime import timedelta
import pandas as pd
from itables import show
import itables.options as opt

In [5]:
asset_symbol = "MSFT"

In [7]:
############ Asset related
asset_ask_data = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': asset_symbol,
        'secType': 'STK',
        'exchange': 'SMART',
        'currency': 'USD',
    }),
    barSizeSetting='15 mins',
    durationStr='2 M',
    whatToShow = 'ASK'
)
asset_bid_data = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': asset_symbol,
        'secType': 'STK',
        'exchange': 'SMART',
        'currency': 'USD',
    }),
    barSizeSetting='15 mins',
    durationStr='2 M',
    whatToShow = 'BID'
)

In [8]:
ask_df = asset_ask_data['hst_dta'][['timestamp', 'low']].rename(columns={'low': 'ask_low'})
bid_df = asset_bid_data['hst_dta'][['timestamp', 'high']].rename(columns={'high': 'bid_high'})

merged_df = ask_df.merge(bid_df, on='timestamp')
merged_df['mid_price'] = (merged_df['ask_low'] + merged_df['bid_high']) / 2
pd.set_option('display.max_rows', None)
#print(merged_df)

############ Set initial parameters
#Rolling Std of Mid-Price Returns
merged_df['log_return'] = np.log(merged_df['mid_price'] / merged_df['mid_price'].shift(1))
window = 30  # Example: 60 periods
merged_df['rolling_volatility'] = merged_df['log_return'].rolling(window=window).std()
merged_df.dropna(subset=['rolling_volatility'], inplace=True)
pd.set_option('display.max_rows', None)
print(merged_df)

               timestamp  ask_low  bid_high  mid_price  log_return  \
30   2025-03-05 10:30:00   391.32    393.05    392.185    0.000153   
31   2025-03-05 10:45:00   391.38    393.28    392.330    0.000370   
32   2025-03-05 11:00:00   390.23    392.43    391.330   -0.002552   
33   2025-03-05 11:15:00   390.69    392.07    391.380    0.000128   
34   2025-03-05 11:30:00   390.70    392.43    391.565    0.000473   
35   2025-03-05 11:45:00   390.39    392.65    391.520   -0.000115   
36   2025-03-05 12:00:00   392.12    393.34    392.730    0.003086   
37   2025-03-05 12:15:00   393.13    394.73    393.930    0.003051   
38   2025-03-05 12:30:00   394.62    395.23    394.925    0.002523   
39   2025-03-05 12:45:00   394.31    396.91    395.610    0.001733   
40   2025-03-05 13:00:00   394.60    396.92    395.760    0.000379   
41   2025-03-05 13:15:00   395.40    397.13    396.265    0.001275   
42   2025-03-05 13:30:00   394.38    395.49    394.935   -0.003362   
43   2025-03-05 13:4

In [10]:
import pandas as pd
import numpy as np

# ===== TUNABLE PARAMETERS =====
gamma = 10
tolerance_bps = 17  # Increased to 4%
initial_inventory = -1
volatility_multiplier = 5  # Reduced to make spreads more reasonable
min_time_diff = 1e-6
# ==============================

df = merged_df.copy()
inventory = initial_inventory
cash_position = 1_000_000.0  # Starting with $1,000,000
reservation_prices = []
optimal_spreads = []
blotter = []
ledger = []

# Precompute constant term in optimal spread
constant_spread_term = (2 / gamma) * np.log(1 + 1 / gamma)
last_fill_time = pd.to_datetime(df.iloc[0]['timestamp'])

# === Main Loop ===
for i, row in df.iterrows():
    current_time = pd.to_datetime(row['timestamp'])
    st = row['mid_price']
    ask = row['ask_low']
    bid = row['bid_high']

    variance = (row['rolling_volatility'] * volatility_multiplier) ** 2
    time_diff = (current_time - last_fill_time).total_seconds() / 60
    time_in_market = max(time_diff, min_time_diff)

    rt = st - gamma * inventory * variance * time_in_market
    reservation_prices.append(rt)

    spread = gamma * variance * time_in_market + constant_spread_term
    optimal_spreads.append(spread)

    tolerance = (tolerance_bps / 10000) * st

    trade = None
    fill_price = None

    # Relaxed trade logic
    if rt <= ask + tolerance:
        trade = 'BUY'
        fill_price = ask
        inventory += 1
        cash_position -= fill_price
        last_fill_time = current_time

    elif rt >= bid - tolerance:
        trade = 'SELL'
        fill_price = bid
        inventory -= 1
        cash_position += fill_price
        last_fill_time = current_time

    # Inventory rebalancing logic
    if inventory >= 10:
        rebalance_qty = inventory  # sell entire position
        inventory -= rebalance_qty
        fill_price = bid  # use bid for sell
        cash_position += bid * rebalance_qty
        trade = 'SELL'
        last_fill_time = current_time

    # Portfolio value = cash + inventory × mid price
    portfolio_value = cash_position + inventory * st
    unrealized_pnl = inventory * (st - fill_price) if fill_price is not None else 0
    total_pnl = portfolio_value  # cash + inventory * mid_price

    # Record trade to blotter
    if trade and fill_price is not None:
        blotter.append({
            'timestamp': current_time,
            'trade': trade,
            'fill_price': round(fill_price, 2),
            'reservation_price': round(rt, 2),
            'inventory': inventory,
            'cash_position': round(cash_position, 3),
            'portfolio_value': round(portfolio_value, 3)
        })

        ledger.append({
            'timestamp': current_time,
            'inventory': inventory,
            'cash_position': round(cash_position, 3),
            'mid_price': round(st, 2),
            'portfolio_value': round(portfolio_value, 3),
            'unrealized_pnl': round(unrealized_pnl, 3)
        })

# Assign to main df
df['reservation_price'] = reservation_prices
df['optimal_spread'] = optimal_spreads

# Final outputs
blotter_df = pd.DataFrame(blotter)
ledger_df = pd.DataFrame(ledger)

# Display results
if not blotter_df.empty:
    print("=== Blotter ===")
    print(blotter_df[['timestamp', 'trade', 'fill_price', 'inventory', 'cash_position']])
else:
    print("⚠️ No trades were executed. Check tolerance_bps or spread settings.")

if not ledger_df.empty:
    print("\n=== Ledger ===")
    print(ledger_df[['timestamp', 'inventory', 'cash_position', 'mid_price', 'portfolio_value']])
else:
    print("⚠️ Ledger is empty.")

# Save to CSV
ledger_df.to_csv("ledger.csv", index=False)


=== Blotter ===
              timestamp trade  fill_price  inventory  cash_position
0   2025-03-05 11:15:00  SELL      392.07         -2     1000392.07
1   2025-03-05 12:00:00  SELL      393.34         -3     1000785.41
2   2025-03-05 12:30:00   BUY      394.62         -2     1000390.79
3   2025-03-05 13:30:00  SELL      395.49         -3     1000786.28
4   2025-03-05 14:15:00   BUY      398.52         -2     1000387.76
5   2025-03-05 14:45:00   BUY      398.84         -1      999988.92
6   2025-03-05 15:30:00   BUY      399.96          0      999588.96
7   2025-03-06 12:15:00   BUY      395.26          1      999193.70
8   2025-03-06 12:30:00   BUY      395.54          2      998798.16
9   2025-03-06 13:00:00   BUY      396.93          3      998401.23
10  2025-03-06 13:15:00   BUY      396.98          4      998004.25
11  2025-03-06 13:30:00   BUY      395.96          5      997608.29
12  2025-03-06 13:45:00   BUY      396.32          6      997211.97
13  2025-03-06 14:00:00   BUY   

In [ ]:
#Alpha and Beta calculation

benchmark_data = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': 'SPX',
        'secType': 'IND',
        'exchange': 'CBOE',
        'currency': 'USD',
    }),
    barSizeSetting='15 mins',
    durationStr='2 M',
)

benchmark_df = pd.DataFrame(benchmark_data['hst_dta'])
benchmark_df['timestamp'] = pd.to_datetime(benchmark_df['timestamp'])
benchmark_df.set_index('timestamp', inplace=True)
benchmark_df_resampled = benchmark_df.reindex(ledger_df['timestamp'], method='nearest')
benchmark_df_resampled['close'] = benchmark_df_resampled['close'].interpolate(method='linear')

close_array = benchmark_df_resampled['close'].values
bench_log_ratios = np.log(close_array[1:] / close_array[:-1])
bench_log_ratios = np.insert(bench_log_ratios, 0, 0)

portfolio_values = ledger_df['portfolio_value'].values
asset_log_ratios = np.log(portfolio_values[1:] / portfolio_values[:-1])
asset_log_ratios = np.insert(asset_log_ratios, 0, 0)

x = sm.add_constant(bench_log_ratios)
y = asset_log_ratios
model = sm.OLS(y,x).fit()
alpha, beta = model.params

print(alpha)
print(beta)

In [31]:
# Get the volatilises

benchmark_volatility_daily = np.std(bench_log_ratios)
benchmark_volatility_annualized = benchmark_volatility_daily * np.sqrt(252)
asset_volatility_daily = np.std(asset_log_ratios)
asset_volatility_annualized = asset_volatility_daily * np.sqrt(252)

print(benchmark_volatility_annualized)
print(asset_volatility_annualized)


0.07183856080115263
0.00022642370194626593


In [35]:
# Get the means

asset_geometric_mean = np.exp(np.mean((asset_log_ratios)))
asset_arithmetic_mean = np.mean(asset_log_ratios)
benchmark_geometric_mean = np.exp(np.mean((bench_log_ratios)))
benchmark_arithmetic_mean = np.mean(bench_log_ratios)

print(benchmark_geometric_mean)
print(benchmark_arithmetic_mean)
print(asset_geometric_mean)
print(asset_arithmetic_mean)

0.9999813518481299
-1.8648325749104094e-05
1.0000012432731633
1.2432723904057614e-06


In [37]:
# Sharpe Ratio
asset_log_ratios = np.log(ledger_df['portfolio_value'].values[1:] / ledger_df['portfolio_value'].values[:-1])
asset_log_ratios = np.insert(asset_log_ratios, 0, 0)

asset_mean_return = np.mean(asset_log_ratios)
asset_volatility_daily = np.std(asset_log_ratios)
asset_volatility_annualized = asset_volatility_daily * np.sqrt(252)

sharpe_ratio = asset_mean_return / asset_volatility_annualized
print(f"Sharpe Ratio: {sharpe_ratio}")


Sharpe Ratio: 0.005490910976717492


In [40]:
# 2. Average Return per Trade
blotter_df['portfolio_value_prev'] = blotter_df['portfolio_value'].shift(1)
blotter_df['trade_pnl'] = blotter_df['portfolio_value'] - blotter_df['portfolio_value_prev']
blotter_df = blotter_df.dropna(subset=['trade_pnl'])
# Calculate average return per trade
avg_return_per_trade = blotter_df['trade_pnl'].mean()
print(f"Average Return per Trade: {avg_return_per_trade}")
num_trades = len(blotter_df)
start_time = blotter_df['timestamp'].iloc[0]
end_time = blotter_df['timestamp'].iloc[-1]
time_period_days = (end_time - start_time).days
time_period_years = time_period_days / 365.25
avg_trades_per_year = num_trades / time_period_years
print(f"Average Number of Trades per Year: {avg_trades_per_year}")

Average Return per Trade: 1.2608092485550546
Average Number of Trades per Year: 4357.810344827586
